In [26]:
import keras
import pandas as pd
import numpy as np

In [27]:
basic_bmodel = keras.models.load_model("/Volumes/Flash/basic_bmodel/")
comb_bmodel = keras.models.load_model("/Volumes/Flash/comb_bmodel/")

In [28]:
basic_norm_bounds = pd.read_csv("/Volumes/Flash/basic_bmodel/data/norm_bounds_basic.csv")
comb_norm_bounds = pd.read_csv("/Volumes/Flash/comb_bmodel/data/norm_bounds_combined.csv")
basic_bounds = (basic_norm_bounds["M10X"], basic_norm_bounds["M10Y"])
comb_bounds = (comb_norm_bounds["M10X"], comb_norm_bounds["M10Y"])
print(comb_bounds)

(0   -15
1    15
Name: M10X, dtype: int64, 0     0
1    40
Name: M10Y, dtype: int64)


In [29]:
xmin = -15
xmax = 15
ymin = 0
ymax = 40

def normalize(x, y):
    ret = np.zeros(2)
    ret[0] = (x - xmin) / (xmax - xmin)
    ret[1] = (y - ymin) / (ymax - ymin)
    ret = ret.reshape(1,-1)
    return ret

In [39]:
pmin = 95
pmax = 121

def rescale(p_list):
    ret = []
    for p in p_list:
        ret.append((p * (pmax - pmin)) + pmin)
    return ret

In [40]:
xc = [[-9,-7,-5,-3,-1,1,3,5,7,9],
      [-9,-7,-5,-3,-1,1,3,5,7,9],
      [-7,-5,-3,-1,1,3,5,7],
      [-5,-3,-1,1,3,5],
      [-3,-1,1,3],
      [-1,1]]
yc = 25

In [43]:
simple_controller = {"basic": {}, "comb": {}}
for i, row in enumerate(xc):
    for x in row:
        y = yc + i*2
        x_in = normalize(x,y)
        basic_pred = basic_bmodel.predict(x_in)
        basic_final = rescale(list(basic_pred[0]))
        simple_controller["basic"][(x,y)] = basic_final
        comb_pred = comb_bmodel.predict(x_in)
        comb_final = rescale(list(comb_pred[0]))
        simple_controller["comb"][(x,y)] = comb_final
        print((x,y))
        print(basic_final)
        print(comb_final)
    

(-9, 25)
[101.1165723502636, 109.99031257629395, 100.69990667700768, 100.85667139291763]
[101.10601526498795, 106.58479034900665, 101.25661557912827, 101.5618206858635]
(-7, 25)
[100.44536930322647, 103.63160914182663, 100.88497549295425, 102.68599200248718]
[100.63723054528236, 101.09634578227997, 101.38243120908737, 106.0349897146225]
(-5, 25)
[98.7752859890461, 98.99141275882721, 100.61817905306816, 109.72915363311768]
[100.93545377254486, 100.83283939957619, 101.30535972118378, 102.63546568155289]
(-3, 25)
[99.69420337677002, 99.73016154766083, 100.83649751543999, 101.47033563256264]
[100.85943919420242, 100.69595488905907, 101.10590600967407, 101.30652433633804]
(-1, 25)
[99.52471196651459, 99.59916985034943, 101.17942088842392, 101.23659241199493]
[100.85166889429092, 100.69280740618706, 101.09704858064651, 101.30769437551498]
(1, 25)
[99.57550019025803, 99.6781494319439, 101.33118698000908, 101.1546625494957]
[100.84389781951904, 100.68965992331505, 101.08819076418877, 101.30886

In [48]:
print(len(simple_controller["comb"]))
print(len(simple_controller["basic"]))

40
40


In [49]:
import pickle

In [50]:
pickle.dump( simple_controller, open( "/Volumes/Flash/controllers/simple_controller.p", "wb" ) )